In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U bitsandbytes
print("Dependencies installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; plat

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TrainingArguments
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-11 18:56:19.154195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749668179.368872      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749668179.427126      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
dataset = load_dataset("blended_skill_talk")

def format_dialog(example):
    return {
        "prompt": example["previous_utterance"],
        "response": example["free_messages"][-1] if example["free_messages"] else ""
    }

formatted_dataset = dataset['train'].map(format_dialog)
formatted_dataset = formatted_dataset.train_test_split(test_size=0.1)

README.md:   0%|          | 0.00/8.99k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length = 256,
    dtype = torch.float16,
    load_in_4bit = True  # Use 4-bit for max speed/memory savings
)
tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [5]:
# STEP 3: Apply LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 10,
    lora_alpha = 16,
    #lora_dropout = 0.10,
    target_modules = ["q_proj", "v_proj"],
    bias = "none",
    random_state=26917,
    use_gradient_checkpointing='unsloth', # Recommended by Unsloth
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.6.2 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from unsloth import FastLanguageModel
from unsloth.trainer import SFTTrainer
from transformers import TrainingArguments

# STEP 4: Safe Tokenization for batched=True
def tokenize(example):
    input_texts = [f"<s>Human: {prompt}\n<|Assistant|>: {response}</s>"
                   for prompt, response in zip(example["prompt"], example["response"])]
    return tokenizer(input_texts, padding="max_length", truncation=True, max_length=256)

# Apply tokenization
tokenized_data = formatted_dataset.map(
    tokenize,
    batched=True,
    remove_columns=formatted_dataset["train"].column_names
)

# Convert to PyTorch tensors
tokenized_data.set_format(type="torch", columns=["input_ids", "attention_mask"])


# STEP 5 (Training Arguments)
training_args = TrainingArguments(
    output_dir="./unsloth-tinyllama-trial1",
    per_device_train_batch_size=100,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
)

# STEP 6 (Trainer setup with correct dataset fields)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
)

# STEP 7 (Train)
trainer.train()

# STEP 8 (Save the model)
model.save_pretrained("best_model_trial1")

print("SFT MODEL TRAINED")

Map:   0%|          | 0/4337 [00:00<?, ? examples/s]

Map:   0%|          | 0/482 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,337 | Num Epochs = 1 | Total steps = 11
O^O/ \_/ \    Batch size per device = 100 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (100 x 4 x 1) = 400
 "-____-"     Trainable parameters = 1,408,000/617,014,272 (0.23% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
# STEP 6: Evaluate both base and SFT models
from evaluate import load
import nltk
import torch

nltk.download("punkt")
bleu = load("bleu")

eval_prompts = [
    "How do I start a small business?",
    "Tell me a joke about programmers.",
    "How can I improve my communication skills?",
    "What is the capital of France?",
    "Explain machine learning like I'm 5.",
    "Give me a fun fact about space.",
    "How do I bake a cake?",
    "What's the best way to stay motivated?",
    "What are some good habits to build?",
    "Tell me about the importance of sleep."
]

references = [
    "Start by identifying your niche, writing a business plan, and registering your company legally.",
    "Why do programmers prefer dark mode? Because light attracts bugs!",
    "Practice active listening, use open body language, and be clear and concise.",
    "Paris is the capital of France.",
    "Machine learning is like teaching a computer to learn from experience, like a child learning from mistakes.",
    "One day on Venus is longer than a year on Venus!",
    "Mix flour, eggs, sugar, and butter, then bake at 350°F for 30 minutes.",
    "Set small goals, reward yourself, and stay consistent even when motivation dips.",
    "Wake up early, exercise, read regularly, and stay organized.",
    "Sleep helps the brain recover, improve memory, and maintain health."
]

def evaluate_model(model, tokenizer, prompts, refs):
    model.eval()
    outputs = []
    for prompt in prompts:
        inputs = tokenizer(f"<s>Human: {prompt}\nAssistant:", return_tensors="pt").to(model.device)
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=100)
        response = tokenizer.decode(output[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
        outputs.append(response)

    results = bleu.compute(predictions=outputs, references=[[r] for r in refs])

    print("\nSample Responses:")
    for i, (prompt, response, ref) in enumerate(zip(prompts, outputs, refs)):
        print(f"\nPrompt {i+1}: {prompt}")
        print(f"Model Output: {response}")
        print(f"Reference    : {ref}")

    return results["bleu"]


In [ ]:
# STEP 7: Load both models and evaluate
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    torch_dtype=torch.float16
)

sft_model = AutoModelForCausalLM.from_pretrained(
    "best_model_trial1",
    device_map="auto",
    torch_dtype=torch.float16
)

models = [
    ("Base Model", base_model),
    ("SFT Model", sft_model)
]

print("Evaluating models on unseen prompts...\n")
for name, model in models:
    print(f"\n--- {name} ---")
    score = evaluate_model(model, tokenizer, eval_prompts, references)
    print(f"\n{name} BLEU Score: {score:.4f}")

ALL TRIALS WERE DONE ON THIS NOTEBOOK
DETAILS OF TRIALS ARE DISCUSSED IN THE PROJECT REPORT 
THIS IS THE SAME CODE FILE USED ACROSS THE TRIAL BY DIFFERENT GROUP MEMBERS WITH DIFFERENT PARAMETERS